# © Artur Czarnecki. All rights reserved.
# Integrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.

# Notebook 10 — E2E: User Long-Term Memory (LTM)
This notebook validates the engine-integrated user long-term memory path end-to-end:
- persistence (profile → JSON snapshot),
- modifications (edit + soft delete),
- semantic retrieval via `SessionManager.search_user_longterm_memory(...)`,
- engine injection via `_step_user_longterm_memory` (SYSTEM message),
- multi-session simulation (Session 1 → Session 2) without refactors.

In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

In [2]:
from intergrax.llm_adapters.base import LLMAdapter, LLMAdapterRegistry, LLMProvider
from intergrax.runtime.drop_in_knowledge_mode.config import RuntimeConfig
from intergrax.runtime.drop_in_knowledge_mode.engine.runtime import DropInKnowledgeRuntime
from intergrax.runtime.drop_in_knowledge_mode.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.drop_in_knowledge_mode.session.session_manager import SessionManager
from intergrax.memory.user_profile_manager import UserProfileManager
from intergrax.memory.stores.in_memory_user_profile_store import InMemoryUserProfileStore
from intergrax.rag.embedding_manager import EmbeddingManager
from intergrax.rag.vectorstore_manager import VSConfig, VectorstoreManager

# ---------------------------------------------------------------------
# Test identifiers
# ---------------------------------------------------------------------
USER_ID = "user_e2e_ltm_001"
SESSION_1 = "sess_e2e_ltm_001"
SESSION_2 = "sess_e2e_ltm_002"

# ---------------------------------------------------------------------
# LLM adapter (real adapter, no wrappers)
# - assumes env is configured (OPENAI_API_KEY etc.)
# ---------------------------------------------------------------------
llm_adapter = LLMAdapterRegistry.create(LLMProvider.OLLAMA)


# ---------------------------------------------------------------------
# Embeddings + vectorstore (real managers)
# Pick providers you actually use in your repo/env.
# ---------------------------------------------------------------------

embed_manager = EmbeddingManager(
    verbose=True,
    provider="ollama",
)

vectorstore_manager = VectorstoreManager(
    config=VSConfig(
        provider="chroma",
        collection_name="intergrax_user_ltm",
    ),
    verbose=True,
)

# ---------------------------------------------------------------------
# Stores
# ---------------------------------------------------------------------
session_store = InMemorySessionStorage()
user_profile_store = InMemoryUserProfileStore()

# ---------------------------------------------------------------------
# Managers
# ---------------------------------------------------------------------
user_profile_manager = UserProfileManager(
    store=user_profile_store,
    embedding_manager=embed_manager,
    vectorstore_manager=vectorstore_manager,
)

session_manager = SessionManager(
    storage=session_store,
    user_profile_manager=user_profile_manager,
)

# ---------------------------------------------------------------------
# Runtime config (LTM enabled, RAG disabled to isolate the feature)
# ---------------------------------------------------------------------
config = RuntimeConfig(
    llm_adapter=llm_adapter,
    enable_user_profile_memory=True,
    enable_org_profile_memory=False,
    enable_user_longterm_memory=True,
    enable_rag=False,
    enable_websearch=False,
    tools_mode="off",
)

runtime = DropInKnowledgeRuntime(
    config=config,
    session_manager=session_manager,
    ingestion_service=None,
    context_builder=None,
    rag_prompt_builder=None,
    websearch_prompt_builder=None,
    history_prompt_builder=None,
)

print("BOOTSTRAP OK")
print("USER_ID:", USER_ID)
print("SESSION_1:", SESSION_1)
print("SESSION_2:", SESSION_2)



[intergraxVectorstoreManager] Initialized provider=chroma, collection=intergrax_user_ltm
[intergraxVectorstoreManager] Existing count: 0
BOOTSTRAP OK
USER_ID: user_e2e_ltm_001
SESSION_1: sess_e2e_ltm_001
SESSION_2: sess_e2e_ltm_002
